# Sentinel-2 Sunglint Finder with Earth Engine Python API

This notebook shows how to find Sentinel-2 scenes over sunglint geometry using the Earth Engine Python API.  
Adjust the `bbox`, `start_date`, `end_date`, and other parameters as needed.


In [ ]:
# 1. Install and import Earth Engine Python API
# Uncomment the following line if running in Colab or a new environment:
# !pip install earthengine-api
%load_ext autoreload
%autoreload 2

import ee
import math
import geopandas as gpd
from shapely import Point
import geemap
from sunglint_raster import add_b8a_b3_sgi, add_abascal_sun_glint_bands


# Initialize the Earth Engine client. 
# If running for the first time, you may need to authenticate:
# ee.Authenticate()
ee.Initialize()
print('Earth Engine initialized.')


In [ ]:
import pandas as pd
infra = pd.read_csv(r"C:\Users\ebeva\SkyTruth\methane\nonoise_SAR_fixed_infrastructure.csv")

# 1. Define the Gulf of Mexico bounding box
min_lon, min_lat = -98.0, 24.0
max_lon, max_lat = -80.5, 31.0



In [ ]:
from shapely.geometry import Polygon, Point
import geopandas as gpd

# Define the polygon as a shapely Polygon (longitude, latitude order)
gulf_poly = Polygon([
    (-90.384631, 23.859054),
    (-90.384631, 28.645609),
    (-85.68112, 28.645609),
    (-85.68112, 23.859054),
    (-90.384631, 23.859054)
])

# Create GeoDataFrame from infra DataFrame
gdf = gpd.GeoDataFrame(
    infra,
    geometry=[Point(xy) for xy in zip(infra['lon'], infra['lat'])],
    crs="EPSG:4326"
)

# Filter gdf by whether points are within the polygon
gdf_gulf = gdf[gdf.geometry.within(gulf_poly)]
gdf_gulf.plot()

In [ ]:

# infra.sample(1)
row = gdf_gulf.sample(1).iloc[0]
stationary_infra = ee.Geometry.Point([row['lon'], row['lat']])
Map = geemap.Map()
Map.addLayer(stationary_infra)
Map.centerObject(stationary_infra, 8)
Map

In [ ]:
def get_s2_collection(start_date, end_date, stationary_infra, cloud_percent=20):
    return (
        ee.ImageCollection('COPERNICUS/S2_SR')
        .filterDate(start_date, end_date)
        .filterBounds(stationary_infra)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_percent))
    )

s2 = get_s2_collection('2018-01-01', '2025-12-01', stationary_infra=stationary_infra)

In [ ]:
sorted_s2 = s2.sort('MEAN_SOLAR_ZENITH_ANGLE').toList(10)
idx = 2
high_glint_fract = ee.Image(sorted_s2.get(idx))

In [ ]:
print_s2_metadata_angles(high_glint_fract)

In [ ]:
# Visualize RGB - .visualize(bands=['B4', 'B3', 'B2'], min=0, max=4000)
# Visualize single band {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'red']}

Map.addLayer(stationary_infra,{'color': 'red'}, 'Stationary')
Map.centerObject(stationary_infra, 9)
Map